# NEP → GAA Budget Anomaly Detection

This notebook analyzes how proposed government budgets (NEP) change after approval (GAA).
It identifies unusual budget adjustments to support transparency and public understanding.

NEP = Proposed Budget  
GAA = Approved Budget


## Background

Government budgets go through a review process before final approval.
While proposed (NEP) and approved (GAA) budgets are public, understanding how and where changes happen is difficult.

This notebook focuses on:
- Identifying large or unusual changes
- Comparing regions fairly
- Avoiding false alarms
- Explaining results in simple language


## Dataset Overview

The dataset contains budget records with:
- Budget type (NEP or GAA)
- Fiscal year
- Budget amount
- Agency and region
- Budget classification and object codes

Only records from a selected agency are analyzed.


## Key Definitions

- NEP (National Expenditure Program): Proposed government budget
- GAA (General Appropriations Act): Final approved budget
- Budget Drift: Difference between NEP and GAA
- Anomaly: A budget adjustment that is unusually large or inconsistent with peers or history

Anomalies indicate items that may require review, not wrongdoing.


## Data Preparation

We separate NEP and GAA records and aggregate them to a comparable level
to ensure fair comparison.


In [1]:
import pandas as pd

df = pd.DataFrame()

fiscl_years = [2020, 2021, 2022, 2023, 2024, 2025, 2026]

for year in fiscl_years:
    df_year = pd.read_parquet(f'cleaned_budget_{year}.parquet', engine='pyarrow')
    df = pd.concat([df, df_year], ignore_index=True)

In [2]:
pd.set_option('display.max_columns', None)
df.head()

,budget_id,budget_type,fiscal_year,budget_amount,budget_description,funding_code,funding_source,department_code,department_name,abbreviation,agency_code,full_agency_code,agency_name,org_code,org_name,region_code,region_description,prexc_fpap_id,uacs_object_code,uacs_classification,uacs_sub_class,uacs_group,uacs_object_name,uacs_sub_object_name
0,GAA-2020-0000000001,GAA,2020,0.0,General Administration and Support,nan,None,None,None,None,None,None,None,nan,None,nan,None,100000000000000,nan,None,None,None,None,None
1,GAA-2020-0000000002,GAA,2020,543636.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010101001,Expenses,Personnel Services,Salaries and Wages,Salaries and Wages - Regular,Basic Salary - Civilian
2,GAA-2020-0000000003,GAA,2020,60383.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010102000,Expenses,Personnel Services,Salaries and Wages,Salaries and Wages - Casual/Contractual,Salaries and Wages - Casual/Contractual
3,GAA-2020-0000000004,GAA,2020,16824.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010201001,Expenses,Personnel Services,Other Compensation,Personal Economic Relief Allowance (PERA),PERA - Civilian
4,GAA-2020-0000000005,GAA,2020,11772.0,General management and supervision,01101101,Regular Agency Fund - General Fund - New Gener...,01,Congress of the Philippines (CONGRESS),CONGRESS,001,01001,Senate,010010000000,Senate,13,National Capital Region (NCR),100000100001000,5010202000,Expenses,Personnel Services,Other Compensation,Representation Allowance (RA),Representation Allowance (RA)


In [3]:
nep = df[df['budget_type'] == 'NEP']
gaa = df[df['budget_type'] == 'GAA']

# Aggregate to comparable level
group_cols = [
    'fiscal_year',
    'department_code',
    'department_name',
    'full_agency_code',
    'agency_name',
    'region_code',
    'region_description',
    'budget_description',
    'prexc_fpap_id'
]

nep_agg = nep.groupby(group_cols)['budget_amount'].sum().reset_index()
gaa_agg = gaa.groupby(group_cols)['budget_amount'].sum().reset_index()

merged = nep_agg.merge(
    gaa_agg,
    on=group_cols,
    suffixes=('_nep', '_gaa'),
    how='outer'
)

In [ ]:
merged[merged['budget_description']=='Construction of Multi-Purpose Building, Barangay Kamuning, Puerto Princesa City, Palawan']#.sample(6)#[merged['prexc_fpap_id']=='300104215736000']#.sample(6)

In [4]:
merged['unapproved_budget'] = merged['budget_amount_gaa'].isna()
merged['inserted_budget'] = merged['budget_amount_nep'].isna()
merged['budget_amount_gaa'] = merged['budget_amount_gaa'].fillna(0)
merged['budget_amount_nep'] = merged['budget_amount_nep'].fillna(0)

## Measuring Budget Changes

We calculate:
- Absolute change (difference in amount)
- Percentage change (relative adjustment)

Small changes are common and expected.


In [5]:
# Calculate drift
merged['abs_change'] = merged['budget_amount_gaa'] - merged['budget_amount_nep']
merged['pct_change'] = merged['abs_change'] / merged['budget_amount_nep'].replace(0, 1)

## Interpreting Budget Adjustments

- NEP > GAA: Budget reduction
- NEP < GAA: Budget increase
- Minimal difference: Approved as proposed


In [6]:
merged

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,budget_description,prexc_fpap_id,budget_amount_nep,budget_amount_gaa,unapproved_budget,inserted_budget,abs_change,pct_change
0,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),Administration of Personnel Benefits,100000100002000,213401.0,213401.0,False,False,0.0,0.000000
1,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),General management and supervision,100000100001000,2216813.0,2559213.0,False,False,342400.0,0.154456
2,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),Legislation of Laws and Other Related Activities,310100100001000,2353949.0,2483549.0,False,False,129600.0,0.055056
3,2020,01,Congress of the Philippines (CONGRESS),01001,Senate,13,National Capital Region (NCR),Senate Relocation,100000200001000,2877000.0,4387000.0,False,False,1510000.0,0.524852
4,2020,01,Congress of the Philippines (CONGRESS),01002,Senate Electoral Tribunal,13,National Capital Region (NCR),Adjudication of Electoral Contests involving M...,310100100001000,110724.0,110724.0,False,False,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
257284,2026,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),Administration of Personnel Benefits,100000100002000,13935.0,0.0,True,False,-13935.0,-1.000000
257285,2026,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),General Management and Supervision,100000100001000,1024214.0,0.0,True,False,-1024214.0,-1.000000
257286,2026,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),Membership Promotion,310100100003000,72631.0,0.0,True,False,-72631.0,-1.000000
257287,2026,40,Department of Migrant Workers (DMW),40002,Overseas Workers Welfare Administration,13,National Capital Region (NCR),Training and Scholarship Grant,310100100001000,36581.0,0.0,True,False,-36581.0,-1.000000


In [7]:
def classify_adjustment(row):
    if row['unapproved_budget']:
        return "Unapproved Budget"
    elif row['inserted_budget']:
        return "Inserted Budget"
    elif row['pct_change'] > 0.05:
        return "Budget Increase"
    elif row['pct_change'] < -0.05:
        return "Budget Reduction"
    else:
        return "No Significant Change"

merged['adjustment_type'] = merged.apply(classify_adjustment, axis=1)

## Anomaly Detection Strategy (Summary)

This project uses multiple anomaly signals to reduce false positives.
A budget item is flagged only when it meets more than one condition:

- Meaningful budget change
- Unusual compared to peers
- Inconsistent with historical behavior

Final decisions are based on a combined anomaly score.


### Threshold Selection

Thresholds were chosen to balance sensitivity and false positives.
They are conservative by design and intended for review support, not enforcement.


In [8]:
merged['anomaly_threshold'] = (
    (merged['pct_change'].abs() > 0.3) |
    (merged['abs_change'].abs() > 50_000_000)
)

### Z-Score (Statistical Outliers)
Logic:

Compare each item to its peers (same year, same classification).

In [9]:
merged['z_score'] = merged.groupby(
    ['fiscal_year', 'budget_description']
)['pct_change'].transform(
    lambda x: (x - x.mean()) / x.std(ddof=0)
)

merged['anomaly_zscore'] = merged['z_score'].abs() > 2

### Regional Comparison

Regions with no or minimal budget changes are treated as normal outcomes
of the approval process and are not flagged as anomalies.

In [10]:
merged['region_mean'] = merged.groupby(
    ['fiscal_year', 'budget_description']
)['pct_change'].transform('mean')

merged['region_std'] = merged.groupby(
    ['fiscal_year', 'budget_description']
)['pct_change'].transform('std')

MIN_CHANGE = 0.05

merged['region_anomaly'] = (
    (merged['pct_change'].abs() > MIN_CHANGE) &
    ((merged['pct_change'] - merged['region_mean']).abs() >
     2 * merged['region_std'])
)

### Historical Consistency Check

Logic:

Compare current drift against historical average drift.

In [11]:
merged['historical_mean'] = merged.groupby(
    ['full_agency_code', 'region_code', 'budget_description']
)['pct_change'].transform('mean')

merged['historical_std'] = merged.groupby(
    ['full_agency_code', 'region_code', 'budget_description']
)['pct_change'].transform('std')

merged['historical_anomaly'] = (
    (merged['pct_change'] - merged['historical_mean']).abs() >
    2 * merged['historical_std']
)

## Final Anomaly Scoring

In [12]:
merged['anomaly_score'] = (
    merged['anomaly_threshold'].astype(int) +
    merged['anomaly_zscore'].astype(int) +
    merged['historical_anomaly'].astype(int) +
    merged['region_anomaly'].astype(int)
)

merged['is_anomaly'] = merged['anomaly_score'] >= 2


Interpretation

Score 0–1: Normal

Score 2–3: Needs review

Score 4: High-risk anomaly

## Natural Language Explanation

In [13]:
def explain(row):
    reasons = []
    if row['anomaly_threshold']:
        reasons.append("large budget change")
    if row['anomaly_zscore']:
        reasons.append("statistically unusual compared to peers")
    if row['historical_anomaly']:
        reasons.append("inconsistent with historical trends")
    if row['region_anomaly']:
        reasons.append("differs from other regions")
    return '' if not reasons else "Flagged due to: " + ", ".join(reasons)

merged['explanation'] = merged.apply(explain, axis=1)

In [ ]:
# Save the results
merged.to_parquet('detected_anomaly_project.parquet')

## Reviewing Sample Results

The following samples show:
- Random non-anomalous records
- Random detected anomalies

This helps validate that the detection logic behaves as expected.


In [14]:
# Sample some non-anomalies
merged[~merged['is_anomaly']].sample(5)

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,budget_description,prexc_fpap_id,budget_amount_nep,budget_amount_gaa,unapproved_budget,inserted_budget,abs_change,pct_change,adjustment_type,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_std,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
71663,2021,18,Department of Public Works and Highways (DPWH),18001,Office of the Secretary,09,Region IX - Zamboanga Peninsula,"NRJ leading to Basay Falls, Barangay Culasian,...",300203101883000,0.0,7500.0,False,True,7500.0,7500.0,Inserted Budget,True,NaN,False,7500.0,NaN,False,10023.166667,1625.490746,False,1,False,Flagged due to: large budget change
32834,2020,18,Department of Public Works and Highways (DPWH),18001,Office of the Secretary,11,Region XI - Davao,"Construction of footbridges, Brgy. Ilang, Dava...",300119200402000,0.0,3000.0,False,True,3000.0,3000.0,Inserted Budget,True,NaN,False,3000.0,NaN,False,3000.000000,NaN,False,1,False,Flagged due to: large budget change
211916,2025,18,Department of Public Works and Highways (DPWH),18001,Office of the Secretary,02,Region II - Cagayan Valley,Construction of Bridge and Installation of RC ...,300221107778000,0.0,10000.0,False,True,10000.0,10000.0,Inserted Budget,True,NaN,False,10000.0,NaN,False,10000.000000,NaN,False,1,False,Flagged due to: large budget change
232178,2025,18,Department of Public Works and Highways (DPWH),18001,Office of the Secretary,13,National Capital Region (NCR),Pangasinan-Zambales Road to Judge Jose De Vene...,310204102892000,0.0,70000.0,False,True,70000.0,70000.0,Inserted Budget,True,NaN,False,70000.0,NaN,False,70000.000000,NaN,False,1,False,Flagged due to: large budget change
243013,2026,05,Department of Agriculture (DA),05001,Office of the Secretary,06,Region VI - Western Visayas,Modernization and Strengthening of Production ...,200000100015000,61981.0,0.0,True,False,-61981.0,-1.0,Unapproved Budget,True,NaN,False,-1.0,0.0,False,-1.000000,NaN,False,1,False,Flagged due to: large budget change


In [15]:
# Sample some detected anomalies
merged[(merged['is_anomaly']) & (~merged['unapproved_budget'])  & (merged['abs_change'] >1000000)].sample(7)

,fiscal_year,department_code,department_name,full_agency_code,agency_name,region_code,region_description,budget_description,prexc_fpap_id,budget_amount_nep,budget_amount_gaa,unapproved_budget,inserted_budget,abs_change,pct_change,adjustment_type,anomaly_threshold,z_score,anomaly_zscore,region_mean,region_std,region_anomaly,historical_mean,historical_std,historical_anomaly,anomaly_score,is_anomaly,explanation
131833,2023,17,Department of National Defense (DND),17006,Veterans Memorial Medical Center,13,National Capital Region (NCR),In-Patient care,310100100001000,0.0,1459181.0,False,True,1459181.0,1.459181e+06,Inserted Budget,True,NaN,False,1.459181e+06,NaN,False,2.084543e+05,5.515186e+05,True,2,True,"Flagged due to: large budget change, inconsist..."
164217,2024,05,Department of Agriculture (DA),05003,Bureau of Fisheries and Aquatic Resources,13,National Capital Region (NCR),"Monitoring, control and surveillance",310200100001000,477771.0,2377771.0,False,False,1900000.0,3.976801e+00,Budget Increase,True,3.872983,True,2.485500e-01,9.942002e-01,True,3.310666e-01,1.651253e+00,True,4,True,"Flagged due to: large budget change, statistic..."
155913,2023,20,Department of Social Welfare and Development (...,20001,Office of the Secretary,07,Region VII - Central Visayas,Pantawid Pamilyang Pilipino Program (Implement...,310100100001000,0.0,7494027.0,False,True,7494027.0,7.494027e+06,Inserted Budget,True,0.511234,False,6.413155e+06,2.183580e+06,False,1.070575e+06,2.832476e+06,True,2,True,"Flagged due to: large budget change, inconsist..."
48474,2020,35,Budgetary Support to Government Corporations (...,35005,National Irrigation Administration,13,National Capital Region (NCR),"Small Irrigation Project (SIP), Nationwide",310204200001000,0.0,5023478.0,False,True,5023478.0,5.023478e+06,Inserted Budget,True,3.872983,True,3.139664e+05,1.255870e+06,True,1.004695e+06,2.246568e+06,False,3,True,"Flagged due to: large budget change, statistic..."
125256,2023,05,Department of Agriculture (DA),05002,Agricultural Credit Policy Council,13,National Capital Region (NCR),Agro-Industry Modernization Credit and Financi...,310100100001000,0.0,2773403.0,False,True,2773403.0,2.773403e+06,Inserted Budget,True,NaN,False,2.773403e+06,NaN,False,3.962001e+05,1.048248e+06,True,2,True,"Flagged due to: large budget change, inconsist..."
196950,2024,20,Department of Social Welfare and Development (...,20001,Office of the Secretary,13,National Capital Region (NCR),Sustainable Livelihood Program,310100100002000,4081032.0,6090151.0,False,False,2009119.0,4.923066e-01,Budget Increase,True,3.872983,True,3.076916e-02,1.230767e-01,True,2.491299e-01,6.753516e-01,False,3,True,"Flagged due to: large budget change, statistic..."
210016,2025,16,Department of Labor and Employment (DOLE),16009,Technical Education and Skills Development Aut...,13,National Capital Region (NCR),"Promotion, Development, Implementation, Monito...",310300100002000,2088722.0,3726222.0,False,False,1637500.0,7.839722e-01,Budget Increase,True,3.872983,True,4.899826e-02,1.959931e-01,True,6.454708e+05,1.290939e+06,False,3,True,"Flagged due to: large budget change, statistic..."


In [25]:
pd.set_option('display.max_rows', None)
df[ (df['department_name'] == 'Department of Public Works and Highways (DPWH)') & (df['budget_type'] == 'GAA') ].groupby(['fiscal_year','budget_type','department_name','uacs_object_code','uacs_sub_object_name'])['budget_amount'].sum().reset_index().sort_values(by=['budget_type','budget_amount','fiscal_year','budget_type'],ascending=False,).head(100)

,fiscal_year,budget_type,department_name,uacs_object_code,uacs_sub_object_name,budget_amount
444,2025,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,585883409.0
370,2024,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,492815014.0
223,2022,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,470335084.0
295,2023,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,459391292.0
150,2021,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,414564314.0
371,2024,GAA,Department of Public Works and Highways (DPWH),5060403002,Flood Control Systems,349332963.0
445,2025,GAA,Department of Public Works and Highways (DPWH),5060403002,Flood Control Systems,346620462.0
76,2020,GAA,Department of Public Works and Highways (DPWH),5060403001,Road Networks,338246670.0
296,2023,GAA,Department of Public Works and Highways (DPWH),5060403002,Flood Control Systems,280667834.0
224,2022,GAA,Department of Public Works and Highways (DPWH),5060403002,Flood Control Systems,208903148.0


In [ ]:
df

## Limitations

- This analysis does not determine intent or correctness
- Results should be reviewed alongside official documents
- Some budget changes may be policy-driven or emergency-related


## Summary

This notebook demonstrates a transparent and explainable approach
to identifying unusual budget adjustments between NEP and GAA.

Future improvements include:
- Multi-year trend analysis
- Interactive dashboards
- Integration with Snowflake


In [ ]:
merged[['department_name', 'budget_amount_nep', 'budget_amount_gaa']].groupby('department_name').sum().reset_index().sort_values(by='budget_amount_nep', ascending=False).head(5)

In [ ]:
import matplotlib.pyplot as plt

sample = merged[['department_name', 'budget_amount_nep', 'budget_amount_gaa']].groupby('department_name').sum().reset_index().sort_values(by='budget_amount_nep', ascending=False).head(12)

plt.figure()
plt.bar(sample['department_name'], sample['budget_amount_nep'], label='NEP')
plt.bar(sample['department_name'], sample['budget_amount_gaa'], bottom=sample['budget_amount_nep'], label='GAA')
plt.title('NEP vs GAA Budget Comparison')
plt.ylabel('Budget Amount')
plt.xticks(rotation=45, ha='right')
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
sample

In [ ]:
import numpy as np

sample = merged[['department_name', 'budget_amount_nep', 'budget_amount_gaa']].groupby('department_name').sum().reset_index().sort_values(by='budget_amount_nep', ascending=False).head(12)

x = np.arange(len(sample))
width = 0.35

plt.figure()
plt.bar(x - width/2, sample['budget_amount_nep'], width, label='NEP')
plt.bar(x + width/2, sample['budget_amount_gaa'], width, label='GAA')

plt.xticks(x, sample['department_name'], rotation=45, ha='right')
plt.ylabel('Budget Amount')
plt.title('NEP (Proposed) vs GAA (Approved)')
plt.legend()
plt.tight_layout()
plt.show()
